In [1]:
import os
import pandas as pd
import json
from utils import normalize_trait, judge_and_remove_biased_features

In [2]:
source_root = "/home/techt/AI4Science_gold_standard/data/preprocessed/gold"
dest_root = "./gold_subset"

## check Xena data omission

In [ ]:
s_ls = []
for t in os.listdir(dest_root):
    s_path = os.path.join(source_root, t)
    if not os.path.isdir(s_path):
        f_ls = os.listdir(os.path.join(dest_root, t))
        if len(f_ls) == 0 or (len(f_ls) == 1 and f_ls[0].endswith('.json')):
            pass
        else:
            # print(f"Strange. Dir {s_path} does not exist!")
            s_ls.append(t)
    else:
        for f in os.listdir(s_path):
            if f.endswith('Xena.csv'):
                source_fpath = os.path.join(s_path, f)
                if os.path.isfile(os.path.join(dest_root, t, 'Xena.csv')):
                    print(f"{os.path.join(dest_root, t, 'Xena.csv')} exist corresponding to {source_fpath}")
                else:
                    print(f"No! {os.path.join(dest_root, t, 'Xena.csv')} not exist corresponding to {source_fpath}")

In [ ]:
sorted(s_ls)

## Fix table abnormality
delete table with missing genetic data, and unify trait name as header. Creat json files 'cohort_info.json'.

In [ ]:
def org_normalize_trait(trait):
    trait = '-'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

In [ ]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]
all_org_nm_traits = [org_normalize_trait(at) for at in all_traits]

In [ ]:
sorted(all_nm_traits)

In [ ]:
len(set(all_nm_traits))

In [ ]:
sorted(all_org_nm_traits)

In [ ]:
# 检查路径和列名里的trait_name，进行normalize
# 没有gene的table去掉
# 创建或修改json文件

In [ ]:
for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_org_nm_traits:
            n_trait = all_nm_traits[all_org_nm_traits.index(trait)]
            os.rename(os.path.join(dest_root, trait), os.path.join(dest_root, n_trait))
            print(f"'{trait}' trait has been newly normalized to {n_trait} in the dir name")
            trait = n_trait
        else:
            print(f"!!!!!!!!!!!!!!!!!!'{trait}' strange trait")
            continue
    trait_dir = os.path.join(dest_root, trait)
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            df = pd.read_csv(fp)
            if 'Unnamed: 0' in df.columns:
                print(f"REDUNDANT COLUMN in '{fp}'")
                df = df.drop(columns = ['Unnamed: 0'], errors="ignore")
            header = df.columns[0]
            if header not in all_nm_traits:
                if header in all_traits:
                    nm_trait = all_nm_traits[all_traits.index(header)]
                    print(f"The header {header} should be replaced by {nm_trait} in '{fn}'")
                    df.columns = [nm_trait] + df.columns[1:].tolist()
                    df.to_csv(fp, index=False)
                    print('This cohort updated')
                else:
                    print(f"Strange header {header}")
            # elif header == "Lupus-(Systemic-Lupus-Erythematosus":
            #     df.columns = ["Lupus (Systemic Lupus Erythematosus)"] + df.columns[1:].tolist()
            #     df.to_csv(fp, index=False)
            #     print("The cohort for 'Lupus' updated")

            if len(df.columns) <= 4:
                print(f"NO GENE DATA in '{fp}'")
                os.remove(fp)
                print("File removed successfully")


In [ ]:
dd = pd.read_csv("./trait_related_genes.csv")
for i in range(len(dd)):
    tr = dd.iat[i, 1]
    if tr in all_traits:
        dd.iat[i, 1] = normalize_trait(tr)
        print(f"{tr} normalized to {dd.iat[i, 1]}")
    else:
        assert tr in ['Age', 'Gender']

In [ ]:
dd = pd.read_csv("./trait_related_genes.csv")
for i in range(len(dd)):
    g = dd.iat[i, 2]
    print(g)

In [ ]:
dd.to_csv("./trait_related_genes.csv", index=False)

In [ ]:
import pandas as pd

In [3]:
de = pd.read_csv("./trait_condition_pairs.csv")
for i in range(len(de)):
    w = de.iat[i, 1]
    z = de.iat[i, 2]
    de.iat[i, 1] = normalize_trait(w)
    de.iat[i, 2] = normalize_trait(z)

In [4]:
de

,Unnamed: 0,Trait,Condition,IoU
0,0,Lung_Cancer,Age,0.348796
1,2,Pancreatic_Cancer,Endometriosis,0.290725
2,3,Endometriosis,Pancreatic_Cancer,0.290725
3,4,Breast_Cancer,Endometriosis,0.287714
4,5,Endometriosis,Breast_Cancer,0.287714
...,...,...,...,...
995,3764,Multiple_Chemical_Sensitivity,Age,0.000000
996,3770,Multiple_Chemical_Sensitivity,Hypertension,0.000000
997,3783,Aniridia,Anxiety_disorder,0.000000
998,3790,Prostate_Cancer,Anxiety_disorder,0.000000


In [6]:
de

,Unnamed: 0,Trait,Condition,IoU
0,0,Lung_Cancer,Age,0.348796
1,2,Pancreatic_Cancer,Endometriosis,0.290725
2,3,Endometriosis,Pancreatic_Cancer,0.290725
3,4,Breast_Cancer,Endometriosis,0.287714
4,5,Endometriosis,Breast_Cancer,0.287714
...,...,...,...,...
995,3764,Multiple_Chemical_Sensitivity,Age,0.000000
996,3770,Multiple_Chemical_Sensitivity,Hypertension,0.000000
997,3783,Aniridia,Anxiety_disorder,0.000000
998,3790,Prostate_Cancer,Anxiety_disorder,0.000000


In [7]:
de.to_csv("./trait_condition_pairs.csv", index=False)

In [5]:
dd.to_csv("./trait_condition_pairs.csv", index=False)

NameError: name 'dd' is not defined

In [ ]:
de = pd.read_csv("./trait_condition_pairs.csv")
for i in range(len(de)):
    tr = de.iat[i, 1]
    if tr in all_traits:
        dd.iat[i, 1] = normalize_trait(tr)
        print(f"{tr} normalized to {dd.iat[i, 1]}")
    else:
        assert tr in ['Age', 'Gender']

In [ ]:
for trait in os.listdir(dest_root):
    if trait != 'Lung-Cancer':
        continue
    else:
        print("Haha lung cancer")
    if trait not in all_nm_traits:
        if trait in all_traits:
            print(f"'{trait}' trait needs to be normalized ")
        else:
            print(f"'{trait}' strange trait")
        continue
    trait_dir = os.path.join(dest_root, trait)
    meta = dict()
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            cohort = fn[:-4]
            df = pd.read_csv(fp)
            header = df.columns[0]
            assert header in all_traits
            trait_type = 'binary' if len(df[header].unique()) <= 2 else 'continuous'
            # May mistakenly judge (28, 28, 28, 28, ..., 100) as unbiased
            # May later try len(unique_values) == 2 and set(unique_values) <= {0, 1}
            trait_biased, _ = judge_and_remove_biased_features(df, header, trait_type)
            has_age = 'Age' in df.columns or 'age' in df.columns
            has_gender = 'Gender' in df.columns or 'gender' in df.columns
            sample_size = len(df)
            if not trait_biased:
                meta[cohort] = {"is_usable": True, "is_available": True, "is_biased": False, "has_age": has_age, "has_gender": has_gender, "sample_size": sample_size, "note": ""}
    js_path = os.path.join(trait_dir, "cohort_info.json")
    if os.path.exists(js_path):
        os.remove(js_path)
    with open(js_path, 'w') as file:
        json.dump(meta, file)


In [ ]:
from utils import read_json_to_dataframe, filter_and_rank_cohorts
b = read_json_to_dataframe('gold_subset/Creutzfeldt-Jakob-Disease/cohort_info.json')
b

In [ ]:
b.empty

In [1]:
import pandas as pd
import ast
from utils import normalize_gene_symbols

def normalize_and_save_genes(file_path, output_file_path):
    """Read a CSV file about gene-trait association, normalize the related genes for each trait, remove None values,
    and save the modified table to a new file."""
    # Read the CSV file into a DataFrame
    related_gene_df = pd.read_csv(file_path)
    related_gene_df = related_gene_df.loc[:, ['Trait', 'Related_Genes']].set_index('Trait')

    # Normalize gene symbols and remove None values
    def normalize_genes(genes):
        # Convert the string representation of the list to a list
        gene_list = ast.literal_eval(genes)
        # Normalize gene symbols using the provided function
        normalized_genes = normalize_gene_symbols(gene_list)
        # Filter out None values
        return [gene for gene in normalized_genes if gene is not None]

    # Apply normalization to each row in the 'Related_Genes' column
    related_gene_df['Related_Genes'] = related_gene_df['Related_Genes'].apply(normalize_genes)

    # Save the modified DataFrame to a new CSV file
    related_gene_df.to_csv(output_file_path)

# Example usage:
# normalize_and_save_genes('./trait_related_genes_org.csv', './trait_related_genes.csv')

In [2]:
normalize_and_save_genes('./trait_related_genes_org.csv', './trait_related_genes.csv')